# Tick data

For optimum results this notebook should be run during the Forex trading session.

In [5]:
from ib_async import *

util.startLoop()

ib = IB()
ib.connect("127.0.0.1", 4001, clientId=99)


<IB connected to 127.0.0.1:4001 clientId=99>

Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1102, reqId -1: Connectivity between IB and Trader Workstation has been restored - data maintained. The following farms are connected: usfuture; usfarm; secdefil. The following farms are not connected: ushmds.
Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1102, reqId -1: Connectivity between IB and Trader Workstation has been restored - data maintained. The following farms are connected: usfuture; usfarm; secdefil. The following farms are not connected: ushmds.
Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1102, reqId -1: Connectivity between IB and Trader Workstation has been restored - data maintained. The following farms are connected: usfutur

In [6]:
ib.reqAccountSummary()
pos = ib.positions()
print(pos)


[Position(account='U2340948', contract=Stock(conId=4762, symbol='BA', exchange='NYSE', currency='USD', localSymbol='BA', tradingClass='BA'), position=254.0, avgCost=203.21112205), Position(account='U2340948', contract=Stock(conId=47486780, symbol='EDV', exchange='ARCA', currency='USD', localSymbol='EDV', tradingClass='EDV'), position=650.0, avgCost=77.7080769), Position(account='U2340948', contract=Stock(conId=229726228, symbol='RSX', exchange='VALUE', currency='USD', localSymbol='RSX', tradingClass='RSX'), position=100.0, avgCost=6.889977), Position(account='U10394496', contract=Option(conId=637079300, symbol='HYG', lastTradeDateOrContractMonth='20240621', strike=73.0, right='P', multiplier='100', currency='USD', localSymbol='HYG   240621P00073000', tradingClass='HYG'), position=50.0, avgCost=12.44685), Position(account='U10394496', contract=Future(conId=637533450, symbol='NQ', lastTradeDateOrContractMonth='20240920', multiplier='20', currency='USD', localSymbol='NQU4', tradingClass='

In [7]:
pos = ib.positions(account="U10394496")


In [11]:
pos[1].contract


Future(conId=637533450, symbol='NQ', lastTradeDateOrContractMonth='20240920', multiplier='20', currency='USD', localSymbol='NQU4', tradingClass='NQ')

### Streaming tick data

Create some Forex contracts:

In [2]:
contracts = [
    Forex(pair) for pair in ("EURUSD", "USDJPY", "GBPUSD", "USDCHF", "USDCAD", "AUDUSD")
]
ib.qualifyContracts(*contracts)

eurusd = contracts[0]


Request streaming ticks for them:

In [3]:
for contract in contracts:
    ib.reqMktData(contract, "", False, False)


Wait a few seconds for the tickers to get filled.

In [4]:
ticker = ib.ticker(eurusd)
ib.sleep(2)

ticker


Ticker(contract=Forex('EURUSD', conId=12087792, exchange='IDEALPRO', localSymbol='EUR.USD', tradingClass='EUR.USD'), time=datetime.datetime(2024, 5, 1, 22, 41, 20, 200620, tzinfo=datetime.timezone.utc), minTick=1e-05, bid=1.07173, bidSize=2000000.0, ask=1.07177, askSize=7000000.0, prevAsk=1.07175, prevAskSize=6000000.0, high=1.0721, low=1.0708, close=1.0713)

The price of Forex ticks is always nan. To get a midpoint price use ``midpoint()`` or ``marketPrice()``.

The tickers are kept live updated, try this a few times to see if the price changes:

In [5]:
ticker.marketPrice()


1.07175

The following cell will start a 30 second loop that prints a live updated ticker table.
It is updated on every ticker change.

In [6]:
from IPython.display import display, clear_output
import pandas as pd

df = pd.DataFrame(
    index=[c.pair() for c in contracts],
    columns=["bidSize", "bid", "ask", "askSize", "high", "low", "close"],
)


def onPendingTickers(tickers):
    for t in tickers:
        df.loc[t.contract.pair()] = (
            t.bidSize,
            t.bid,
            t.ask,
            t.askSize,
            t.high,
            t.low,
            t.close,
        )
        clear_output(wait=True)
    display(df)


ib.pendingTickersEvent += onPendingTickers
ib.sleep(30)
ib.pendingTickersEvent -= onPendingTickers


,bidSize,bid,ask,askSize,high,low,close
EURUSD,2000000.0,1.07177,1.0718,2000000.0,1.0721,1.0708,1.0713
USDJPY,1000000.0,155.215,155.224,1000000.0,155.355,154.21,154.45
GBPUSD,1000000.0,1.25363,1.25368,2000000.0,1.2538,1.25165,1.2528
USDCHF,1000000.0,0.91643,0.91652,2000000.0,0.91705,0.9156,0.9158
USDCAD,3000000.0,1.373,1.37307,2000000.0,1.3741,1.373,1.3739
AUDUSD,1000000.0,0.65305,0.65308,1000000.0,0.6531,0.6515,0.6523


New tick data is available in the 'ticks' attribute of the pending tickers.
The tick data will be cleared before the next update.

To stop the live tick subscriptions:

In [8]:
for contract in contracts:
    ib.cancelMktData(contract)


cancelMktData: No reqId found for contract Forex('EURUSD', conId=12087792, exchange='IDEALPRO', localSymbol='EUR.USD', tradingClass='EUR.USD')
cancelMktData: No reqId found for contract Forex('USDJPY', conId=15016059, exchange='IDEALPRO', localSymbol='USD.JPY', tradingClass='USD.JPY')
cancelMktData: No reqId found for contract Forex('GBPUSD', conId=12087797, exchange='IDEALPRO', localSymbol='GBP.USD', tradingClass='GBP.USD')
cancelMktData: No reqId found for contract Forex('USDCHF', conId=12087820, exchange='IDEALPRO', localSymbol='USD.CHF', tradingClass='USD.CHF')
cancelMktData: No reqId found for contract Forex('USDCAD', conId=15016062, exchange='IDEALPRO', localSymbol='USD.CAD', tradingClass='USD.CAD')
cancelMktData: No reqId found for contract Forex('AUDUSD', conId=14433401, exchange='IDEALPRO', localSymbol='AUD.USD', tradingClass='AUD.USD')
cancelMktData: No reqId found for contract Forex('USDJPY', conId=15016059, exchange='IDEALPRO', localSymbol='USD.JPY', tradingClass='USD.JPY')

### Tick by Tick data ###

The ticks in the previous section are time-sampled by IB in order to cut on bandwidth. So with ``reqMktdData`` not every tick from the exchanges is sent. The promise of ``reqTickByTickData`` is to send every tick, just how it appears in the TWS Time & Sales window. This functionality is severly nerfed by a total of just three simultaneous subscriptions, where bid-ask ticks and sale ticks also use up a subscription each.

The tick-by-tick updates are available from ``ticker.tickByTicks`` and are signalled by ``ib.pendingTickersEvent`` or ``ticker.updateEvent``.

In [9]:
ticker = ib.reqTickByTickData(eurusd, "BidAsk")
ib.sleep(2)
print(ticker)

ib.cancelTickByTickData(ticker.contract, "BidAsk")


Ticker(contract=Forex('EURUSD', conId=12087792, exchange='IDEALPRO', localSymbol='EUR.USD', tradingClass='EUR.USD'), time=datetime.datetime(2024, 5, 1, 22, 42, 49, 350050, tzinfo=datetime.timezone.utc), minTick=1e-05, bid=1.07175, bidSize=2000000.0, ask=1.07178, askSize=2000000.0, prevBid=1.07178, prevBidSize=1000000.0, prevAsk=1.0718, prevAskSize=1000000.0, high=1.0721, low=1.0708, close=1.0713, tickByTicks=[TickByTickBidAsk(time=datetime.datetime(2024, 5, 1, 22, 42, 49, 350050, tzinfo=datetime.timezone.utc), bidPrice=1.07175, askPrice=1.07178, bidSize=2000000.0, askSize=2000000.0, tickAttribBidAsk=TickAttribBidAsk(bidPastLow=False, askPastHigh=False))])


### Historical tick data

Historical tick data can be fetched with a maximum of 1000 ticks at a time. Either the start time or the end time must be given, and one of them must remain empty:

In [9]:
import datetime

start = ''
end = datetime.datetime.now()
ticks = ib.reqHistoricalTicks(eurusd, start, end, 1000, 'BID_ASK', useRth=False)

ticks[-1]


HistoricalTickBidAsk(time=datetime.datetime(2019, 12, 31, 17, 5, 34, tzinfo=datetime.timezone.utc), tickAttribBidAsk=TickAttribBidAsk(bidPastLow=False, askPastHigh=False), priceBid=1.1225, priceAsk=1.1226, sizeBid=11000000, sizeAsk=11500000)

In [4]:
ib.reqAccountSummary()


ConnectionError: Not connected

In [10]:
ib.disconnect()
